## Analyse d'ombre sur un établissement de St denis

### Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.affinity import translate
from pathlib import Path
import numpy as np

### Chargement des Bâtiments et Zones Éducatives

In [ ]:
DATA_FOLDER = Path("../data")

saint_denis_path = DATA_FOLDER / "saint_denis_reference_data.gpkg"

ecoles = gpd.read_file(saint_denis_path, layer="bdtopo_education").to_crs(2154)
batiments = gpd.read_file(saint_denis_path, layer="bdtopo_batiment").to_crs(2154)

### Préparation des données pour l'analyse des ombres

In [ ]:

ecole_cible = ecoles.iloc[[6]]  # On prend la 6ème école par exemple

#Vérification de la hauteur 
if "hauteur" in batiments.columns:
    hauteur_ecole = batiments.loc[batiments["cleabs_right__bat"] == ecole_cible["cleabs_right"].iloc[0], "hauteur"]
    if not hauteur_ecole.empty:
        hauteur_ecole = float(hauteur_ecole.iloc[0])
    else:
        raise ValueError("None")
else:
    raise KeyError("None")


batiments["distance"] = batiments.geometry.distance(ecole_cible.geometry.iloc[0])

rayon = 100  #rayon de 100m
batiments_voisins = batiments[batiments["distance"] < rayon].copy()

batiments_voisins["hauteur"] = pd.to_numeric(batiments_voisins["hauteur"], errors="coerce")

batiments_plus_hauts = batiments_voisins[batiments_voisins["hauteur"] > hauteur_ecole]

### Calcul de l'ombre projetée en fonction des 4 jours clés

In [ ]:

saisons = {
    "hiver": 15,       # Soleil bas en hiver (21 décembre)
    "printemps": 45,   # Équinoxe (21 mars)
    "été": 75,         # Soleil haut en été (21 juin)
    "automne": 45      # Équinoxe (21 septembre)
}

jours_cles = ["hiver", "printemps", "été", "automne"]

### Visualisation comprenant les ombres des bâtiment voisins sur les toits (non sur le sol)

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, saison in enumerate(jours_cles):
    angle_solaire = np.radians(saisons[saison])
    ombres_toit = []
    
    for _, row in batiments_plus_hauts.iterrows():
        hauteur_relative = row["hauteur"] - hauteur_ecole
        distance_ombre = hauteur_relative / np.tan(angle_solaire)
        
       
        deplacement_x = distance_ombre * np.cos(np.radians(45))
        deplacement_y = distance_ombre * np.sin(np.radians(45))
        ombre_projetee = translate(row["geometry"], xoff=deplacement_x, yoff=deplacement_y)
        
        
        intersection = batiments_voisins.intersects(ombre_projetee)
        if intersection.any():
            ombres_toit.append(ombre_projetee)
    
    ombre_toit_gdf = gpd.GeoDataFrame(geometry=ombres_toit, crs=2154)
    
    batiments_voisins.plot(ax=axes[i], color="none", edgecolor="black", linewidth=0.8)
    ombre_toit_gdf.plot(ax=axes[i], color="grey", alpha=0.7, label=f"Ombres sur toits - {saison}")
    cx.add_basemap(axes[i], crs=batiments_voisins.crs, source=cx.providers.OpenStreetMap.Mapnik)
    axes[i].set_title(f"Projection des ombres sur les toits - {saison.capitalize()}")

plt.tight_layout()
plt.show()
